In [1]:
# Libraries import 
import pdbufr
import sys
import traceback
 
from math import isnan
from eccodes import *
from ecmwf.opendata import Client
from datetime import datetime, timedelta
from PIL import Image

import os
import birdy
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import requests

from ipywidgets import interact
import ipyleaflet
import ipywidgets as widgets

from Magics import macro as magics
from IPython.display import display
from Magics.macro import *

import warnings
warnings.filterwarnings("ignore")

from tracks_utils import *

In [2]:
df_storms_forecast = create_storms_df()
code = df_storms_forecast.stormIdentifier.unique()[0]

In [31]:
# Function that returns the list of coordinates for the mean forecast track
def mean_forecast_track(df_storm):
    
    # Create 2 dataframe with latitude and logitude coordinates for each ensemble as columns
    members = df_storm.ensembleMemberNumber.unique()
    df_lat_tracks = pd.DataFrame()
    df_lon_tracks = pd.DataFrame()
    df_radius_tracks = pd.DataFrame()
    for member in members:
        df_track = df_storm[df_storm.ensembleMemberNumber == member]
        df_track.reset_index(drop=True, inplace=True)
        df_lat_tracks[f'latitude{member}'] = df_track.latitude
        df_lon_tracks[f'longitude{member}'] = df_track.longitude
        df_radius_tracks[f'radius{member}'] = df_track.effectiveRadiusWithRespectToWindSpeedsAboveThreshold
    # Add date information for the average track
    df_track.reset_index(drop=True, inplace=True)
    start = datetime(df_track.iloc[0]['year'], df_track.iloc[0]['month'], df_track.iloc[0]['day'], df_track.iloc[0]['hour'])
    dates = start + timedelta(hours=6) * (df_track.index+1)
    
    # Cycle through the rows of df_lat_track and df_lon_tracks to compute the average track lat,lon
    mean_track_coord = []
    timesteps = []
    radii = []
    for t in range(len(df_lat_tracks)):
        lat = df_lat_tracks.iloc[t].dropna().to_numpy()
        lon = df_lon_tracks.iloc[t].dropna().to_numpy()
        date = dates[t].strftime("%d-%m-%Y %H:%M")
        radius = df_radius_tracks.iloc[t].mean()
        if len(lat) > 0:
            mean_lat_lon = meanposit(len(lat), lat, lon)
            mean_track_coord.append(mean_lat_lon)
            timesteps.append(date)
            radii.append(radius)
        
    return mean_track_coord, radii, timesteps

In [39]:
df_radius_tracks = pd.DataFrame()
df_track = df_storms_forecast[(df_storms_forecast.stormIdentifier == code) & (df_storms_forecast.ensembleMemberNumber == 1)]
df_radius_tracks[f'radius1'] = df_track.effectiveRadiusWithRespectToWindSpeedsAboveThreshold
df_radius_tracks[f'radius2'] = df_track.effectiveRadiusWithRespectToWindSpeedsAboveThreshold
print(df_radius_tracks)
print(df_radius_tracks.iloc[0].mean())

     radius1   radius2
0   225900.0  225900.0
1   188900.0  188900.0
2   118500.0  118500.0
3   179600.0  179600.0
4        0.0       0.0
5    37000.0   37000.0
6    40700.0   40700.0
7        0.0       0.0
8        0.0       0.0
9        0.0       0.0
10       0.0       0.0
11       0.0       0.0
12  248200.0  248200.0
13  205600.0  205600.0
14  150000.0  150000.0
15  151900.0  151900.0
16   42600.0   42600.0
17   53700.0   53700.0
18   51900.0   51900.0
19   44400.0   44400.0
20       0.0       0.0
21       0.0       0.0
22       0.0       0.0
23       0.0       0.0
24  246300.0  246300.0
25  185200.0  185200.0
26  161100.0  161100.0
27  172200.0  172200.0
28   79600.0   79600.0
29   85200.0   85200.0
30   42600.0   42600.0
31   37000.0   37000.0
32       0.0       0.0
33       0.0       0.0
34       0.0       0.0
35       0.0       0.0
36  220400.0  220400.0
37  201900.0  201900.0
38  198200.0  198200.0
39  170400.0  170400.0
40   98200.0   98200.0
225900.0


In [33]:
locations_f, radii_f, timesteps_f = forecast_tracks_locations(df_f)

In [44]:
df_f = df_storms_forecast[df_storms_forecast.stormIdentifier == code]
df_f.reset_index(drop=True, inplace=True)
initial_lat_lon = (df_f.latitude.iloc[0], df_f.longitude.iloc[0])

# create lists of locations
locations_f, radii_f, timesteps_f = forecast_tracks_locations(df_f)
locations_avg, radii_avg, timesteps_avg = mean_forecast_track(df_f)

# Create the basemap for plotting
tc_track_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.0,
    # scroll_wheel_zoom=True,
)

# Define forecasted tracks polyline element for the map
colours = ["red", "blue", "green", "yellow", "purple", "orange", "cyan", "brown"]
tracks_layer_group = ipyleaflet.LayerGroup()
markers_layer_group = ipyleaflet.LayerGroup()
circles_layer_group = ipyleaflet.LayerGroup(name='Circle of wind speeds > 35 kts')
colour = 0
i = 0
# Cycle on the ensembles of the forecast track
for locs in locations_f[:10]:

    tmtstps = timesteps_f[i]
    radius = radii_f[i]

    # Define the ensemble track polyline for the map
    track = ipyleaflet.Polyline(
        locations=locs,
        color=colours[colour],
        fill=False,
        weight=2,
        # name='Track %.02d' % i,
    )
    # Define the markers element for each position and the circle elements for each radius of the winds exceeding the 35 kts threshold of the cyclone ensemble forecast
    markers = []
    circles = []
    for j in range(len(locs)):
        marker = ipyleaflet.CircleMarker(
            location=locs[j],
            radius=1,
            color=colours[colour],
            popup=widgets.HTML(value=f'<b> {tmtstps[j]} </b>')
        )
        markers.append(marker)
        if radius[j] > 0:
            circle = ipyleaflet.Circle(
                location=locs[j],
                radius=int(radius[j]),
                color="purple",
                fill_color="purple",
            )
            circles.append(circle)
        
    colour += 1
    if colour == len(colours):
        colour = 0

    marker_layer = ipyleaflet.LayerGroup(layers=markers)
    circle_layer = ipyleaflet.LayerGroup(layers=circles)

    tracks_layer_group.add_layer(track)
    markers_layer_group.add_layer(marker_layer)
    circles_layer_group.add_layer(circle_layer)

    i += 1

# Define average forecast polyline for the map
track_avg = ipyleaflet.Polyline(
        locations=locations_avg,
        color="black",
        fill=False,
        weight=3,
        # name='Average Forecast Track',
    )

# Define the markers element and the circles element for each position of the average track
marker_avg = []
circle_avg = []
for avg in range(len(locations_avg)):
    marker = ipyleaflet.CircleMarker(
        location = locations_avg[avg],
        radius=1,
        color="black",
        popup=widgets.HTML(value=f'<b> {timesteps_avg[avg]} </b>')
    )
    if radii_avg[avg] > 0:
        circle = ipyleaflet.Circle(
            location=locations_avg[avg],
            radius=int(radii_avg[avg]),
            color="purple",
            fill_color="purple",
            popup=widgets.HTML(value=f'<b> {radii_avg[avg]*10**(-3)} km </b>')
        )
        circle_avg.append(circle)
    marker_avg.append(marker)

# Add average forecast track to the map
markers_layer_avg = ipyleaflet.LayerGroup(layers=marker_avg)
circles_layer_avg = ipyleaflet.LayerGroup(layers=circle_avg, name='Circle of wind speeds > 35 kts')
layer_group_avg = ipyleaflet.LayerGroup(layers=[track_avg, markers_layer_avg], name='Average Forecast Track')

tc_track_map.add_layer(layer_group_avg)
tc_track_map.add_layer(circles_layer_avg)

# Add forecasted ensemble tracks to the map
layer_group_f = ipyleaflet.LayerGroup(layers=[tracks_layer_group, markers_layer_group], name='Forecasted Ensemble Tracks')

tc_track_map.add_layer(layer_group_f)

# Add circle of winds exceeding 35 kts
# tc_track_map.add_layer(circles_layer_group)

# Add layers widget to the map
layers_control = ipyleaflet.LayersControl()
tc_track_map.add_control(layers_control);

display(tc_track_map)

Map(center=[25.900000000000002, -70.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…